In [1]:
%cd ../BackEnd

c:\Users\Korisnik\Documents\GitHub\CineSense\BackEnd


In [ ]:
from sqlalchemy import insert
from sqlalchemy.dialects.postgresql import insert as pg_insert
from sqlalchemy.exc import SQLAlchemyError

async def insert_genres_with_on_conflict(db, new_genres_for_bulk):

    if not new_genres_for_bulk:
        return  # nothing to do

    try:
        # 1) Build the INSERT statement using the PostgreSQL dialect
        stmt = pg_insert(Genre).values(new_genres_for_bulk)

        # 2) Apply on_conflict_do_nothing to skip duplicates
        stmt = stmt.on_conflict_do_nothing(
            index_elements=['genre_id']  # which column to check for conflict
        )

        # 3) Execute and commit
        await db.execute(stmt)
        await db.commit()
        print(f"Inserted {len(new_genres_for_bulk)} genre(s) with upsert logic (duplicate keys ignored).")

    except SQLAlchemyError as e:
        await db.rollback()
        print(f"Insert failed due to SQLAlchemyError: {e}")






In [7]:
import math

def safe_str(val):

    if isinstance(val, float) and math.isnan(val):
        return None
    return val

In [9]:
import datetime

def parse_date(date_str):
    if not date_str or pd.isna(date_str):
        return None
    try:
        return datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        return None


In [ ]:
import csv
import math
import pandas as pd
import psycopg2
import os

TV_TEMP_CSV = "tv_shows_temp.csv"
# ...similar for GENRES_TEMP_CSV, BRIDGING_TEMP_CSV if needed...

def safe_str(val):
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return ""
    return str(val)

def parse_date(val):
    # Your date parsing or just safe_str if you want the DB to parse "YYYY-MM-DD"
    if pd.isna(val):
        return ""
    return str(val)

def load_tv_copy(csv_path: str, batch_size: int = 10000):
    chunk_iter = pd.read_csv(
        csv_path,
        chunksize=batch_size,
        sep=",",
        encoding="utf-8",
        quotechar='"'
    )

    # Remove old temp file if it exists
    if os.path.exists(TV_TEMP_CSV):
        os.remove(TV_TEMP_CSV)

    # Open file in append mode but using csv.writer
    with open(TV_TEMP_CSV, "w", newline="", encoding="utf-8") as tv_file:
        writer = csv.writer(tv_file, quoting=csv.QUOTE_MINIMAL)

        # Optional: Write header row if you want COPY to read header
        writer.writerow([
            "tmdb_id",
            "name",
            "original_name",
            "overview",
            "tagline",
            "first_air_date",
            "popularity",
            "vote_average",
            "vote_count",
            "poster_path",
            "backdrop_path",
            "type"
        ])

        chunk_count = 0
        for chunk_df in chunk_iter:
            chunk_count += 1
            print(f"Processing chunk #{chunk_count} with {len(chunk_df)} rows...")

            for _, row in chunk_df.iterrows():
                tmdb_id = row["id"]  # or row.get("id")
                out_line = [
                    safe_str(tmdb_id),
                    safe_str(row.get("name")),
                    safe_str(row.get("original_name")),
                    safe_str(row.get("overview")),
                    safe_str(row.get("tagline")),
                    parse_date(row.get("first_air_date")),
                    safe_str(row.get("popularity")),
                    safe_str(row.get("vote_average")),
                    safe_str(row.get("vote_count")),
                    safe_str(row.get("poster_path")),
                    safe_str(row.get("backdrop_path")),
                    safe_str(row.get("type")),
                ]
                writer.writerow(out_line)

    print("Finished writing TV temp CSV file.")

    # Now do the COPY
    conn = psycopg2.connect("dbname=cinesense_db user=admin password=dzuver host=45.155.126.141 port=5432")
    conn.autocommit = False

    try:
        with conn.cursor() as cur:
            print("COPYing tv_shows from temp CSV...")
            copy_sql_tv = """
                COPY tv_shows (
                    tmdb_id,
                    name,
                    original_name,
                    overview,
                    tagline,
                    first_air_date,
                    popularity,
                    vote_average,
                    vote_count,
                    poster_path,
                    backdrop_path,
                    type
                )
                FROM STDIN WITH (FORMAT csv, HEADER true, QUOTE '"', ESCAPE '"')
            """
            with open(TV_TEMP_CSV, "r", encoding="utf-8") as f:
                cur.copy_expert(copy_sql_tv, f)

            conn.commit()
            print("All COPY operations committed successfully.")
    except Exception as e:
        conn.rollback()
        print(f"Error during COPY: {e}")
    finally:
        conn.close()
        print("DB connection closed.")

if __name__ == "__main__":
    load_tv_copy("tv.csv", batch_size=10000)
    print("TV import complete via COPY!")


Processing chunk #1 with 10000 rows...
Processing chunk #2 with 10000 rows...
Processing chunk #3 with 10000 rows...
Processing chunk #4 with 10000 rows...
Processing chunk #5 with 10000 rows...
Processing chunk #6 with 10000 rows...
Processing chunk #7 with 10000 rows...
Processing chunk #8 with 10000 rows...
Processing chunk #9 with 10000 rows...
Processing chunk #10 with 10000 rows...
Processing chunk #11 with 10000 rows...
Processing chunk #12 with 10000 rows...
Processing chunk #13 with 10000 rows...
Processing chunk #14 with 10000 rows...
Processing chunk #15 with 10000 rows...
Processing chunk #16 with 10000 rows...
Processing chunk #17 with 10000 rows...
Processing chunk #18 with 10000 rows...
Processing chunk #19 with 10000 rows...
Processing chunk #20 with 6504 rows...
Finished writing TV temp CSV file.
COPYing tv_shows from temp CSV...
